# Forward model for M/EEG source analysis

This notebook illustrates how to use features of the `stormdb.process`-module to:

* generate a BEM model based on pre-calculated tissue surfaces (_e.g._, inner skull, outer skull and outer skin)
* solve the BEM model (calculate interpolation coefficients for the surface elements based on node values)
* generate a _source space_ of possible dipole locations for a distributed (_e.g._, MNE) estimate
    * based on the cortical sheet (white-matter boundary) extracted using _Freesurfer_ `recon-all`
* generate a _forward operator_ for source modeling, using
    * solved BEM model
    * source space
    * head-to-mri transformation
    * an MEG measurement file containing the locations of the sensors relative to the head

_All computations are performed on the `hyades`-cluster._ The underlying code is based on the implementations found in `mne-python` and all outputs are written in the Neuromag FIF-format.

__NB__ The BEM and source space calculations are independent, so we can use the same `MNEPython`-object to hold the jobs, and submit them together/in parallel (steps 1a and 1b below).

In [ ]:
import os.path as op
from stormdb.process import MNEPython
from stormdb.base import mkdir_p  # mkdir -p

In [ ]:
proj_name = 'MINDLAB2015_MEG-Sememene'
subject = '0010_HXN'  # demo subject
scratch_dir = op.join('/projects', proj_name, 'scratch')
subjects_dir = op.join(scratch_dir, 'fs_subjects_dir')

fwd_dir = op.join(scratch_dir, 'fwd_operators')
mkdir_p(fwd_dir)  # create, if not exists

## Step 1a: BEM creation and solution

The BEM surfaces must be created first (see [this notebook](Generate BEM surfaces from MR.ipynb)). Depending on how many layers are to be used, _conductivity values_ must be specified for the desired number of layers. For example, for a 3-layer model, we might say:

In [ ]:
bem_fname = op.join(subjects_dir, subject, 'bem', subject + '-3LBEM-sol.fif')
conductivity = [0.3, 0.006, 0.3]  # brain, skull, skin
# conductivity = [0.3]  # for a single-layer (inner skull) model

In [ ]:
mp = MNEPython(proj_name)

In [ ]:
mp.prepare_bem_model(subject, bem_fname, subjects_dir=subjects_dir,
                     conductivity=conductivity)

## Step 1b: Setup source space

Take a `Freesurfer` surface (by default the white matter surface), downsample it to not more than 10,000 points per hemisphere, and save it to disk for later use. The `add_dist` can be used to allow later calls to `cortical patch statistics` (computation time is increased substantially, but with the cluster, you don't need to care!).

In [ ]:
src_fname = op.join(subjects_dir, subject, 'bem', subject + '-oct-6-src.fif')

In [ ]:
mp.setup_source_space(subject, src_fname, subjects_dir=subjects_dir, add_dist=True)

### Submit BEM and souce space together

In [ ]:
mp.submit()

In [ ]:
mp.status

## Step 2: Coregistration

See [this notebook](Head-to-MRI transformation for combined MEG and EEG.ipynb) for more details on this manual process.

In [ ]:
trans_dir = op.join(scratch_dir, 'trans')
trans_fname = op.join(trans_dir, '0010_HXN_20151201-trans.fif')  # example case

## Step 3: Putting it all together

The "measurement" file name is one containing the head-to-device transformation obtained during the recording. Note that although we are using raw-type file here, epochs and even an averaged "event" file could be used here (they all inherit the head position information from the continuous recording they originate from).

In [ ]:
meas_fname = op.join(scratch_dir, 'maxfilter/tsss_st10_corr98/0010', 'MMN_block1_raw_tsss.fif')

Define output folder for forward operators and the specific file name:

In [ ]:
fwd_fname = op.join(fwd_dir, '0010_HXN-MMN_block1-fwd.fif')

In [ ]:
mp_fwd = MNEPython(proj_name)  # we could re-use the mp-object from before
mp_fwd.make_forward_solution(meas_fname, trans_fname, bem_fname, src_fname, fwd_fname)

In [ ]:
mp_fwd.submit()

In [ ]:
mp_fwd.status

## (Optional) Step 4: Visualise sensitivity distribution

__NB__ This will pop up a separate window with a 3D rendering of the gradiometer sensitivity pattern on cortex

In [ ]:
%matplotlib qt

In [ ]:
from mne import sensitivity_map, read_forward_solution
fwd = read_forward_solution(fwd_fname, surf_ori=True)

In [ ]:
grad_map = sensitivity_map(fwd, ch_type='grad', mode='fixed')

In [ ]:
brain = grad_map.plot(time_label='Gradiometer sensitivity', subjects_dir=subjects_dir,
                      clim=dict(lims=[0, 50, 100]))